# Install required apt packages

In [ ]:
# @title Install required apt packages
!apt-get update
!apt-get install -y build-essential python3.11 python3.11-dev unzip
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1
!update-alternatives --install /usr/bin/python python /usr/bin/python3.11 1
!python3 --version
!python --version
!echo "-= Done. =-"

# Environment Setup

In [ ]:
# @title Environment Setup
from pathlib import Path
import os

OPTIONS = {}

#USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
INSTALL_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_PULID = True  #@param {type:"boolean"}
INSTALL_INSIGHTFACE = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}

#OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['INSTALL_PULID'] = INSTALL_PULID
OPTIONS['INSTALL_INSIGHTFACE'] = INSTALL_INSIGHTFACE
OPTIONS['INSTALL_COMFYUI_MANAGER'] = INSTALL_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

APPLICATIONS_DIR="/opt/applications"

AI_WORKSPACE = f"{APPLICATIONS_DIR}/AI"
COMFY_WORKSPACE = f"{APPLICATIONS_DIR}/AI/ComfyUI"

!echo "Creating AI workspace (if it doesn't exist)"
os.makedirs(AI_WORKSPACE, exist_ok=True)

%cd $AI_WORKSPACE

![ ! -d $COMFY_WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI $COMFY_WORKSPACE

%cd $COMFY_WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo "-= Updating ComfyUI =-"
  !git pull

!echo "-= Install dependencies =-"
!pip install psutil transformers aiohttp matplotlib-inline
!pip install torch torchvision torchaudio
#!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu124
!pip install color-matcher simpleeval torchsde spandrel kornia xformers

if OPTIONS['INSTALL_PULID']:
  !echo "-= Install PuLID =-"
  !pip install --use-pep517 facexlib

if OPTIONS['INSTALL_INSIGHTFACE']:
  !pip install insightface

if OPTIONS['INSTALL_COMFYUI_MANAGER']:
  %cd custom_nodes
  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $COMFY_WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !pwd
  !echo "-= Install custom nodes dependencies =-"
  ![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && python3 "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py"

!echo "-= Install onnxruntime =-"
# !pip uninstall -y onnxruntime
!pip install onnxruntime-gpu onnxruntime-openvino

!echo "-= Done. =-"

# Download models

In [ ]:
# @title Download models
import os

%pip install huggingface_hub[cli]

!huggingface-cli login --token="<Enter your Huggingface token here>"

APPLICATIONS_DIR="/opt/applications"

AI_WORKSPACE = f"{APPLICATIONS_DIR}/AI"
COMFY_WORKSPACE = f"{APPLICATIONS_DIR}/AI/ComfyUI"

COMFY_MODELS = f"{COMFY_WORKSPACE}/models"

COMFY_MODELS_CHECKPOINTS = f"{COMFY_MODELS}/checkpoints"
COMFY_MODELS_UNET = f"{COMFY_MODELS}/unet"
COMFY_MODELS_CLIP = f"{COMFY_MODELS}/clip"
COMFY_MODELS_VAE = f"{COMFY_MODELS}/vae"
COMFY_MODELS_CONTROLNET = f"{COMFY_MODELS}/controlnet"
COMFY_MODELS_UPSCALE_MODELS = f"{COMFY_MODELS}/upscale_models"
COMFY_MODELS_PULID = f"{COMFY_MODELS}/pulid"
COMFY_MODELS_INSIGHTFACE = f"{COMFY_MODELS}/insightface/models"
COMFY_MODELS_LORAS = f"{COMFY_MODELS}/loras"
COMFY_MODELS_ULTRALYTICS = f"{COMFY_MODELS}/ultralytics"

os.makedirs(COMFY_MODELS_CHECKPOINTS, exist_ok=True)
os.makedirs(COMFY_MODELS_UNET, exist_ok=True)
os.makedirs(COMFY_MODELS_CLIP, exist_ok=True)
os.makedirs(COMFY_MODELS_VAE, exist_ok=True)
os.makedirs(COMFY_MODELS_CONTROLNET, exist_ok=True)
os.makedirs(COMFY_MODELS_UPSCALE_MODELS, exist_ok=True)
os.makedirs(COMFY_MODELS_PULID, exist_ok=True)
os.makedirs(COMFY_MODELS_INSIGHTFACE, exist_ok=True)
os.makedirs(COMFY_MODELS_LORAS, exist_ok=True)
os.makedirs(COMFY_MODELS_ULTRALYTICS, exist_ok=True)

# Checkpoints
%cd $COMFY_MODELS_CHECKPOINTS
# sd1.5
!huggingface-cli download stable-diffusion-v1-5/stable-diffusion-v1-5 v1-5-pruned-emaonly.ckpt
!huggingface-cli download stable-diffusion-v1-5/stable-diffusion-v1-5 v1-5-pruned-emaonly.ckpt --local-dir="."

# FULX1 unets
%cd $COMFY_MODELS_UNET
#FLUX schnell
!huggingface-cli download city96/FLUX.1-schnell-gguf flux1-schnell-Q6_K.gguf
!huggingface-cli download city96/FLUX.1-schnell-gguf flux1-schnell-Q6_K.gguf --local-dir="."
# #FLUX dev
# !huggingface-cli download city96/FLUX.1-dev-gguf flux1-dev-Q6_K.gguf
# !huggingface-cli download city96/FLUX.1-dev-gguf flux1-dev-Q6_K.gguf --local-dir="."

#CLIPs
%cd $COMFY_MODELS_CLIP
!huggingface-cli download comfyanonymous/flux_text_encoders clip_l.safetensors
!huggingface-cli download comfyanonymous/flux_text_encoders clip_l.safetensors --local-dir="."
!huggingface-cli download mcmonkey/google_t5-v1_1-xxl_encoderonly t5xxl_fp8_e4m3fn.safetensors
!huggingface-cli download mcmonkey/google_t5-v1_1-xxl_encoderonly t5xxl_fp8_e4m3fn.safetensors --local-dir="."

#VAEs
%cd $COMFY_MODELS_VAE
!huggingface-cli download black-forest-labs/FLUX.1-schnell ae.safetensors
!huggingface-cli download black-forest-labs/FLUX.1-schnell ae.safetensors --local-dir="."

#Controlnets
%cd $COMFY_MODELS_CONTROLNET
# FLUX controlnets
os.makedirs("./flux", exist_ok=True)
%cd "./flux"
!huggingface-cli download InstantX/FLUX.1-dev-Controlnet-Union diffusion_pytorch_model.safetensors
!huggingface-cli download InstantX/FLUX.1-dev-Controlnet-Union diffusion_pytorch_model.safetensors --local-dir="."

# Upscale models
%cd $COMFY_MODELS_UPSCALE_MODELS
!huggingface-cli download Kim2091/ClearRealityV1 4x-ClearRealityV1.pth
!huggingface-cli download Kim2091/ClearRealityV1 4x-ClearRealityV1.pth --local-dir="."

# PuLID models
%cd $COMFY_MODELS_PULID
!huggingface-cli download guozinan/PuLID pulid_flux_v0.9.0.safetensors
!huggingface-cli download guozinan/PuLID pulid_flux_v0.9.0.safetensors --local-dir="."

# Insightface models
%cd $COMFY_MODELS_INSIGHTFACE
!huggingface-cli download MonsterMMORPG/tools antelopev2.zip
!huggingface-cli download MonsterMMORPG/tools antelopev2.zip --local-dir="."
!unzip -o antelopev2.zip
!rm -rf antelopev2.zip

#Loras
%cd $COMFY_MODELS_LORAS
os.makedirs("./FLUX/alimama-creative", exist_ok=True)
%cd "./FLUX/alimama-creative"
!huggingface-cli download alimama-creative/FLUX.1-Turbo-Alpha diffusion_pytorch_model.safetensors --local-dir="."

#Ultralytics
%cd $COMFY_MODELS_ULTRALYTICS
os.makedirs("./bbox", exist_ok=True)
%cd "./bbox"
!huggingface-cli download Bingsu/adetailer face_yolov8m.pt face_yolov8n.pt face_yolov8n_v2.pt
!huggingface-cli download Bingsu/adetailer face_yolov8m.pt face_yolov8n.pt face_yolov8n_v2.pt --local-dir="."

!echo "-= Done. =-"


#  Install rathole and configure

In [ ]:
# @title Install rathole and configure
import os

RATHOLE_SERVER_IP = "<Enter your rathole server IP>"  #@param {type:"string"}
RATHOLE_SERVER_PORT = 2333 # @param {"type":"number"}
RATHOLE_PATH = "/opt/applications/rathole" # @param {"type":"string"}
COMFYUI_PORT = 54321 # @param {"type":"number"}

os.makedirs(RATHOLE_PATH, exist_ok=True)

%cd $RATHOLE_PATH

!wget https://github.com/rapiz1/rathole/releases/download/v0.5.0/rathole-x86_64-unknown-linux-gnu.zip
!unzip -o rathole-x86_64-unknown-linux-gnu.zip; rm -rf rathole-x86_64-unknown-linux-gnu.zip
!chmod 775 rathole
#Create client configuration
!echo "[client]" > client.toml
!echo "remote_addr = \"$RATHOLE_SERVER_IP:$RATHOLE_SERVER_PORT\"" >> client.toml
!echo "" >> client.toml
!echo "[client.services.comfyuitcpa]" >> client.toml
!echo "type = \"tcp\"" >> client.toml
!echo "token = \"YOUR RATHOLE TOKEN\"" >> client.toml
!echo "local_addr = \"localhost:$COMFYUI_PORT\"" >> client.toml
!echo "-= Done. =-"

# Start Rathole tunnel and ComfyUI

In [ ]:
# @title Start Rathole tunnel and ComfyUI
import subprocess
import threading
import time
import socket
import urllib.request

ratholepath = RATHOLE_PATH

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("Starting Rathole")
  p = subprocess.Popen([ratholepath + "/rathole", "--client", ratholepath + "/client.toml"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  !echo "-= Rathole started =-"

threading.Thread(target=iframe_thread, daemon=True, args=(54321,)).start()

%cd $COMFY_WORKSPACE

#!python3 main.py --gpu-only --listen=0.0.0.0 --port=$COMFYUI_PORT
!python3 main.py --lowvram --listen=0.0.0.0 --port=$COMFYUI_PORT
#!python3 main.py --lowvram --cpu-vae --listen=0.0.0.0 --port=$COMFYUI_PORT

!echo "-= Done. =-"

# Install cloudflare

In [ ]:
# @title Install cloudflare
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!rm -rf cloudflared-linux-amd64.deb
!echo "-= Done. =-"

# Start cloudflare tunnel and ComfyUI

In [ ]:
# @title Start cloudflare tunnel and ComfyUI
import subprocess
import threading
import time
import socket
import urllib.request

COMFYUI_PORT = 54321 # @param {"type":"number"}

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      cf_url = l[l.find("http"):]
      print("This is the URL to access ComfyUI:", cf_url, end='')

threading.Thread(target=iframe_thread, daemon=True, args=(COMFYUI_PORT,)).start()

%cd $COMFY_WORKSPACE

#!python3 main.py --gpu-only --listen=0.0.0.0 --port=$COMFYUI_PORT
!python3 main.py --lowvram --listen=0.0.0.0 --port=$COMFYUI_PORT
#!python3 main.py --lowvram --cpu-vae --listen=0.0.0.0 --port=$COMFYUI_PORT


!echo "-= Done. =-"